In [1]:
import pandas as pd
import requests


# In[3]:


url = "https://api.pbpstats.com/get-totals/nba"
params = {
    "Season": "2023-24",
    "SeasonType": "Regular Season",
    "Type": "Player"
}
response = requests.get(url, params=params)
response_json = response.json()
player_stats = response_json["multi_row_table_data"]
df = pd.DataFrame(player_stats)
print(df.columns)
col = ['Name','Minutes','Points','FG2M', 'FG2A', 'FG3M', 'FG3A', 'TsPct','AssistPoints','AtRimAssists','ShortMidRangeAssists', 'LongMidRangeAssists','Corner3Assists','Arc3Assists','LostBallSteals', 'LiveBallTurnovers', 'BadPassOutOfBoundsTurnovers', 'BadPassTurnovers',
       'DeadBallTurnovers', 'LostBallOutOfBoundsTurnovers', 'LostBallTurnovers', 'StepOutOfBoundsTurnovers', 'Travels', 'Turnovers','OffensiveGoaltends','FTA','OffPoss',
 'DefPoss','TotalPoss']
df[col].to_csv('wowy/player_small.csv',index = False)
#df[col].to_csv('2024/player_tracking/player_small.csv',index = False)
df.to_csv('wowy/player_large.csv',index = False)



Index(['EntityId', 'TeamId', 'Name', 'ShortName', 'RowId', 'TeamAbbreviation',
       'SecondsPlayed', 'GamesPlayed', 'Minutes', 'PlusMinus',
       ...
       'PeriodOTFouls5Minutes', '3SecondViolations', 'OffensiveGoaltends',
       'Corner3PctBlocked', 'Transition Take Fouls', 'DefensiveGoaltends',
       'HeaveMakes', 'Clear Path Fouls', 'Period3Fouls5Minutes',
       'Period1Fouls3Minutes'],
      dtype='object', length=249)


In [3]:

for col in df.columns:
    print(col)

EntityId
TeamId
Name
ShortName
RowId
TeamAbbreviation
SecondsPlayed
GamesPlayed
Minutes
PlusMinus
OffPoss
DefPoss
PenaltyOffPoss
PenaltyDefPoss
SecondChanceOffPoss
TotalPoss
AtRimFGM
AtRimFGA
SecondChanceAtRimFGM
SecondChanceAtRimFGA
PenaltyAtRimFGM
PenaltyAtRimFGA
ShortMidRangeFGM
ShortMidRangeFGA
LongMidRangeFGM
LongMidRangeFGA
Corner3FGM
Corner3FGA
SecondChanceCorner3FGM
SecondChanceCorner3FGA
PenaltyCorner3FGM
PenaltyCorner3FGA
Arc3FGM
Arc3FGA
SecondChanceArc3FGM
SecondChanceArc3FGA
PenaltyArc3FGM
PenaltyArc3FGA
FG2M
FG2A
FG3M
FG3A
FtPoints
Points
OpponentPoints
SecondChanceFG2M
SecondChanceFG2A
SecondChanceFG3M
SecondChanceFG3A
SecondChanceFtPoints
SecondChancePoints
PenaltyFG2M
PenaltyFG2A
PenaltyFG3M
PenaltyFG3A
PenaltyFtPoints
PenaltyPoints
PtsAssisted2s
PtsUnassisted2s
PtsAssisted3s
PtsUnassisted3s
NonHeaveArc3FGA
NonHeaveArc3FGM
Fg2aBlocked
Fg3aBlocked
TwoPtAssists
ThreePtAssists
Assists
Arc3Assists
Corner3Assists
AtRimAssists
ShortMidRangeAssists
LongMidRangeAssists
AssistPo